In [1]:
import pandas as pd
import matplotlib
%matplotlib inline
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from sklearn.model_selection import cross_validate
#from sklearn.model_selection import cross_val_score
import pickle
#from  models.populate_dbbikes_database import *
import matplotlib.dates as mdates

In [2]:
from sqlalchemy import create_engine
USER = "tim"
PASSWORD = "Ylu3shin123!!"
URI = "dbbikes.cjlp5umzs5hs.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbbikes"
engine = create_engine("mysql+pymysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [3]:
sql = f"""
SELECT id, number, last_update, available_bikes, available_bike_stands FROM stations
GROUP BY id, day(last_update)
ORDER by id, last_update ASC;
"""

In [4]:
bike_df = pd.read_sql_query(sql, engine)

2021-04-09 17:11:11,831 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-04-09 17:11:11,833 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-09 17:11:11,930 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-04-09 17:11:11,931 INFO sqlalchemy.engine.Engine [generated in 0.00061s] {}
2021-04-09 17:11:12,123 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2021-04-09 17:11:12,124 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-09 17:11:12,410 INFO sqlalchemy.engine.Engine 
SELECT id, number, last_update, available_bikes, available_bike_stands FROM stations
GROUP BY id, day(last_update)
ORDER by id, last_update ASC;

2021-04-09 17:11:12,411 INFO sqlalchemy.engine.Engine [raw sql] {}


In [5]:
bike_df['last_update'] = bike_df['last_update'].astype('datetime64[ns]')

#from datetime import datetime
bike_df['last_update'] = pd.to_datetime(bike_df['last_update'])

#bike_df.last_update = datetime.utcfromtimestamp(bike_df.last_update).strftime('%Y-%m-%d %H:%M:%S')
bike_df

id  number         last_update  available_bikes  \
0            1      42 2021-03-16 19:32:50               23   
1            2      30 2021-03-16 19:26:54                4   
2            3      54 2021-03-16 19:28:27                3   
3            4     108 2021-03-16 19:29:25               23   
4            5      56 2021-03-16 19:34:39                6   
...        ...     ...                 ...              ...   
691330  691331      39 2021-04-09 16:06:56               19   
691331  691332      83 2021-04-09 16:03:37               17   
691332  691333      92 2021-04-09 16:05:35                5   
691333  691334      21 2021-04-09 16:07:18               10   
691334  691335      88 2021-04-09 16:05:10                4   

        available_bike_stands  
0                           7  
1                          16  
2                          30  
3                          17  
4                          34  
...                       ...  
691330                      1  
691331                     22  
691332                     35  
691333                     20  
691334                     26  

[691335 rows x 5 columns]

In [6]:
bike_df.dtypes

id                                int64
number                            int64
last_update              datetime64[ns]
available_bikes                   int64
available_bike_stands             int64
dtype: object

In [7]:
bike_df.shape

(691335, 5)

In [8]:
bike_df.tail()

id  number         last_update  available_bikes  \
691330  691331      39 2021-04-09 16:06:56               19   
691331  691332      83 2021-04-09 16:03:37               17   
691332  691333      92 2021-04-09 16:05:35                5   
691333  691334      21 2021-04-09 16:07:18               10   
691334  691335      88 2021-04-09 16:05:10                4   

        available_bike_stands  
691330                      1  
691331                     22  
691332                     35  
691333                     20  
691334                     26

In [9]:
sql = f"""

SELECT * FROM weather

"""

In [10]:
weather_df = pd.read_sql_query(sql, engine)

2021-04-09 17:11:54,998 INFO sqlalchemy.engine.Engine 

SELECT * FROM weather


2021-04-09 17:11:54,999 INFO sqlalchemy.engine.Engine [raw sql] {}


In [11]:
weather_df.dtypes

id                              int64
description                    object
temp                          float64
last_update_weather    datetime64[ns]
dtype: object

In [12]:
# category    
weather_df['description'] = weather_df['description'].astype('category')    

# datetime   
weather_df['last_update_weather'] = weather_df['last_update_weather'].astype('datetime64[ns]')

In [13]:
weather_df.dtypes

id                              int64
description                  category
temp                          float64
last_update_weather    datetime64[ns]
dtype: object

In [14]:
weather_df.shape

(2874, 4)

In [15]:
weather_df.tail()

id    description  temp last_update_weather
2869  2870  broken clouds  7.50 2021-04-07 13:34:08
2870  2871  broken clouds  7.50 2021-04-07 13:40:02
2871  2872  broken clouds  7.41 2021-04-07 13:53:58
2872  2873  broken clouds  7.60 2021-04-07 14:04:43
2873  2874  broken clouds  7.88 2021-04-07 14:16:41

In [16]:
bike_df.sort_values('last_update', inplace=True)
weather_df.sort_values('last_update_weather', inplace=True)

In [17]:
df = pd.merge_asof(bike_df, weather_df, left_on='last_update', right_on='last_update_weather')


In [18]:
df.tail(5)

id_x  number         last_update  available_bikes  \
691330  691279     115 2021-04-09 16:09:21               19   
691331  691270      19 2021-04-09 16:09:29               13   
691332  691284      16 2021-04-09 16:09:30                6   
691333  691257      99 2021-04-09 16:09:36                7   
691334  691318      85 2021-04-09 16:09:37                8   

        available_bike_stands    id_y    description  temp last_update_weather  
691330                     11  2874.0  broken clouds  7.88 2021-04-07 14:16:41  
691331                     17  2874.0  broken clouds  7.88 2021-04-07 14:16:41  
691332                     14  2874.0  broken clouds  7.88 2021-04-07 14:16:41  
691333                     23  2874.0  broken clouds  7.88 2021-04-07 14:16:41  
691334                     27  2874.0  broken clouds  7.88 2021-04-07 14:16:41

In [19]:
df = df.drop(['id_y'], axis=1)

In [20]:
df = df.drop(['id_x'], axis=1)

In [21]:
df.tail(5)

number         last_update  available_bikes  available_bike_stands  \
691330     115 2021-04-09 16:09:21               19                     11   
691331      19 2021-04-09 16:09:29               13                     17   
691332      16 2021-04-09 16:09:30                6                     14   
691333      99 2021-04-09 16:09:36                7                     23   
691334      85 2021-04-09 16:09:37                8                     27   

          description  temp last_update_weather  
691330  broken clouds  7.88 2021-04-07 14:16:41  
691331  broken clouds  7.88 2021-04-07 14:16:41  
691332  broken clouds  7.88 2021-04-07 14:16:41  
691333  broken clouds  7.88 2021-04-07 14:16:41  
691334  broken clouds  7.88 2021-04-07 14:16:41

In [22]:
stations = list(sorted(df.number.unique()))
print(stations)
print(len(stations))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 507]
110


In [23]:
# New column called weekday with the number of the day of the week
df['weekday'] = df['last_update'].dt.dayofweek 
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.weekday.html
df_monday = df[df.weekday == 0]
df_tuesday = df[df.weekday == 1]
df_wednesday = df[df.weekday == 2]
df_thursday = df[df.weekday == 3]
df_friday = df[df.weekday == 4]
df_saturday = df[df.weekday == 5]
df_sunday = df[df.weekday == 6]

In [24]:

# New column called hourly 
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.hour.html
df['hourly'] = df['last_update'].dt.hour

# https://stackoverflow.com/questions/61124647/pandas-how-to-get-dummies-on-time-series-data
#  hours 0 to 23 will be 0 with one hour being 1
#hourly_dummies = pd.get_dummies(df['hourly'], prefix='hour')

hourly_dummies = pd.get_dummies(df['hourly'], prefix='hour')

# https://towardsdatascience.com/pandas-concat-tricks-you-should-know-to-speed-up-your-data-analysis-cd3d4fdfe6dd
df = pd.concat([df, hourly_dummies], axis=1)

# then drop original column - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html


# list of  hour dummies- # https://datatofish.com/list-column-names-pandas-dataframe/
hour_dummies = list(hourly_dummies)

df.drop(['hourly'], axis=1)

number         last_update  available_bikes  available_bike_stands  \
0           76 2021-03-16 19:24:43               10                     28   
1          110 2021-03-16 19:24:44               13                     27   
2           79 2021-03-16 19:24:45                4                     23   
3           48 2021-03-16 19:24:45               12                     28   
4           43 2021-03-16 19:24:52               13                     17   
...        ...                 ...              ...                    ...   
691330     115 2021-04-09 16:09:21               19                     11   
691331      19 2021-04-09 16:09:29               13                     17   
691332      16 2021-04-09 16:09:30                6                     14   
691333      99 2021-04-09 16:09:36                7                     23   
691334      85 2021-04-09 16:09:37                8                     27   

          description  temp last_update_weather  weekday  hour_0  hour_1  ...  \
0                 NaN   NaN                 NaT        1       0       0  ...   
1                 NaN   NaN                 NaT        1       0       0  ...   
2                 NaN   NaN                 NaT        1       0       0  ...   
3                 NaN   NaN                 NaT        1       0       0  ...   
4                 NaN   NaN                 NaT        1       0       0  ...   
...               ...   ...                 ...      ...     ...     ...  ...   
691330  broken clouds  7.88 2021-04-07 14:16:41        4       0       0  ...   
691331  broken clouds  7.88 2021-04-07 14:16:41        4       0       0  ...   
691332  broken clouds  7.88 2021-04-07 14:16:41        4       0       0  ...   
691333  broken clouds  7.88 2021-04-07 14:16:41        4       0       0  ...   
691334  broken clouds  7.88 2021-04-07 14:16:41        4       0       0  ...   

        hour_14  hour_15  hour_16  hour_17  hour_18  hour_19  hour_20  \
0             0        0        0        0        0        1        0   
1             0        0        0        0        0        1        0   
2             0        0        0        0        0        1        0   
3             0        0        0        0        0        1        0   
4             0        0        0        0        0        1        0   
...         ...      ...      ...      ...      ...      ...      ...   
691330        0        0        1        0        0        0        0   
691331        0        0        1        0        0        0        0   
691332        0        0        1        0        0        0        0   
691333        0        0        1        0        0        0        0   
691334        0        0        1        0        0        0        0   

        hour_21  hour_22  hour_23  
0             0        0        0  
1             0        0        0  
2             0        0        0  
3             0        0        0  
4             0        0        0  
...         ...      ...      ...  
691330        0        0        0  
691331        0        0        0  
691332        0        0        0  
691333        0        0        0  
691334        0        0        0  

[691335 rows x 32 columns]

In [25]:
# dataframes based on day of week
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.weekday.html
# df_monday = df[df.weekday == 0]
# df_tuesday = df[df.weekday == 1]
# df_wednesday = df[df.weekday == 2]
# df_thursday = df[df.weekday == 3]
# df_friday = df[df.weekday == 4]
# df_saturday = df[df.weekday == 5]
# df_sunday = df[df.weekday == 6]

df_monday
df_monday.shape

(93981, 8)

In [26]:
print('Number of duplicate (excluding first) rows in the table is: ', df_monday.duplicated().sum())

print('Number of duplicate rows (including first) in the table is:', df_monday[df_monday.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  43575
Number of duplicate rows (including first) in the table is: 85023


In [27]:
df_monday = df_monday.drop_duplicates(keep='last')
# get duplicated row data. Use "keep=False" to mark all duplicates as true
print('Duplicate remaining rows:', df_monday[df_monday.duplicated(keep='last')].shape[0])

Duplicate remaining rows: 0


In [28]:
df_monday
df_monday.shape

(50406, 8)

In [34]:
df_monday

number         last_update  available_bikes  available_bike_stands  \
104698      30 2021-03-22 00:00:03                0                     20   
104700      45 2021-03-22 00:00:05                0                     30   
104702      57 2021-03-22 00:00:06                4                     19   
104704     115 2021-03-22 00:00:07               30                      0   
104706      87 2021-03-22 00:00:20               11                     27   
...        ...                 ...              ...                    ...   
576008      64 2021-04-05 23:59:09                1                     39   
576009      31 2021-04-05 23:59:16               20                      0   
576010      23 2021-04-05 23:59:29                2                     28   
576011      83 2021-04-05 23:59:32               22                     17   
576012     116 2021-04-05 23:59:36               13                     17   

          description  temp last_update_weather  weekday  
104698  broken clouds  8.03 2021-03-21 23:50:03        0  
104700  broken clouds  8.03 2021-03-21 23:50:03        0  
104702  broken clouds  8.03 2021-03-21 23:50:03        0  
104704  broken clouds  8.03 2021-03-21 23:50:03        0  
104706  broken clouds  8.03 2021-03-21 23:50:03        0  
...               ...   ...                 ...      ...  
576008      clear sky -0.15 2021-04-05 23:51:27        0  
576009      clear sky -0.15 2021-04-05 23:51:27        0  
576010      clear sky -0.15 2021-04-05 23:51:27        0  
576011      clear sky -0.15 2021-04-05 23:51:27        0  
576012      clear sky -0.15 2021-04-05 23:51:27        0  

[50406 rows x 8 columns]

In [29]:
df_tuesday


number         last_update  available_bikes  available_bike_stands  \
0           76 2021-03-16 19:24:43               10                     28   
1          110 2021-03-16 19:24:44               13                     27   
2           79 2021-03-16 19:24:45                4                     23   
3           48 2021-03-16 19:24:45               12                     28   
4           43 2021-03-16 19:24:52               13                     17   
...        ...                 ...              ...                    ...   
607400      78 2021-04-06 23:58:52               16                     24   
607401      83 2021-04-06 23:58:53               18                     21   
607402      62 2021-04-06 23:59:20                0                     40   
607403     116 2021-04-06 23:59:20                7                     23   
607404      75 2021-04-06 23:59:25               24                     16   

            description  temp last_update_weather  weekday  
0                   NaN   NaN                 NaT        1  
1                   NaN   NaN                 NaT        1  
2                   NaN   NaN                 NaT        1  
3                   NaN   NaN                 NaT        1  
4                   NaN   NaN                 NaT        1  
...                 ...   ...                 ...      ...  
607400  overcast clouds  2.46 2021-04-06 23:50:29        1  
607401  overcast clouds  2.46 2021-04-06 23:50:29        1  
607402  overcast clouds  2.46 2021-04-06 23:50:29        1  
607403  overcast clouds  2.46 2021-04-06 23:50:29        1  
607404  overcast clouds  2.46 2021-04-06 23:50:29        1  

[94284 rows x 8 columns]

In [30]:
# Get all features 
A = [hour_dummies] 
B = ['description', 'temp', 'last_update']
features = (A + B)

In [31]:
print(features)


[['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23'], 'description', 'temp', 'last_update']


In [32]:
from itertools import chain

print(list(chain.from_iterable(features)))

['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'd', 'e', 's', 'c', 'r', 'i', 'p', 't', 'i', 'o', 'n', 't', 'e', 'm', 'p', 'l', 'a', 's', 't', '_', 'u', 'p', 'd', 'a', 't', 'e']


In [33]:
# # Taking each day individually

# # Monday

# for station in stations:
    
#     # x-axis is the features
    
#     a = df_monday
    
#     x = a.groupby('number').temp
    
#     #y-axis is the available_bikes
#     y = a.groupby('number').available_bikes
    
#     # y = a.available_bikes
#     # This is from https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html    
#     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
#     model = LinearRegression(fit_intercept=False, normalize=False).fit(x_train, y_train)


#     # Write to a pickle file
#     with open('/?/pickles/Monday-' + str(station) + '.pkl', 'wb') as handle:
#         pickle.dump(model, handle, protocol = pickle.HIGHEST_PROTOCOL)

#     # Create vectors of predictions
#     train_predictions = model.predict(X_train)
#     print('Train predictions for station ', station, ": ", train_predictions)

#     test_predictions = model.predict(X_test)
#     print('Test predictions for station ', station, ": ", test_predictions)


#     # R^2 - R-squared is a statistical measure of how close the data are to the fitted regression line.
#     # r2 = metrics.r2_score(y_true, y_predict)
#     r2 = metrics.r2_score(y_test, test_predictions)
#     print(r2)


#     # how well does the model behave compared to a chosen r2 ? 
#     # an r2 of 0.75 indicates that 25% of the variability in the outcome data cannot be explained by the model
#     # initialise to get how many times the model outperforms r2 = 0.75
#     i = 0
#     if r2 > 0.75:
#         i +=1

#     print('Percentage of models that have outperformed R^2 = 0.75:', (i /110) * 100)  # there are 110 stations